# DATA WRANGLING AND PREPARATION

The detailed and complete application of data science to landing data from Spacex rockets to identify the conditions for a successful landing.

In [1]:
#the list of packages
import pandas as pd
import requests
import numpy as np
#from sklearn import linear_model, tree, model_selection, neighbors, svm
#from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import accuracy_score, jaccard_score, f1_score, log_loss
# import matplotlib.pyplot as plt
# import seaborn as sns
# import datetime as dt
from bs4 import BeautifulSoup

In [2]:
pd.set_option('display.max_columns',None) #display all column names in the dataframe
pd.set_option('display.max_colwidth',None) #display all the data in the dataframe

In [3]:
spacex_url = "https://api.spacexdata.com/v4/launches/past"
spacex_req = requests.get(spacex_url) #using the API means we get a JSON file
main_data_df = pd.json_normalize(spacex_req.json()) #basically the read json is json_normalize

The initial data has been imported and now we organise the data to suit our purposes.

In [4]:
spX = ['rocket','payloads','launchpad','success','cores','flight_number','date_utc']
spacex_df = main_data_df[spX]

In [5]:
#The date needs to be normalised and formatted to keep only the date
spacex_df["date"] = pd.to_datetime(spacex_df["date_utc"]).dt.date
spacex_df = spacex_df.drop("date_utc",axis=1)
#We limit the data set to only one instance of core and of payload and 
#and keep only the first feature of both. This means we are concerned with
#landing of the first phase of the rockets, only.

spacex_df = spacex_df[spacex_df.cores.map(len)==1]
spacex_df['cores'] = spacex_df.cores.map(lambda x: x[0])
spacex_df = spacex_df[spacex_df.payloads.map(len)==1]
spacex_df['payloads'] = spacex_df.payloads.map(lambda x: x[0])


<ipython-input-5-441e8f9915e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df["date"] = pd.to_datetime(spacex_df["date_utc"]).dt.date


To identify the number of booster versions in the rockets column, 

In [6]:
print("The number of booster versions: \n",spacex_df.rocket.value_counts() )

The number of booster versions: 
 5e9d0d95eda69973a809d1ec    122
5e9d0d95eda69955f709d1eb      4
Name: rocket, dtype: int64


So, we need further information to identify the varieties of the booster versions and we are able to do this using another data set.

In [7]:
booster_url = "https://api.spacexdata.com/v4/rockets"
temp_req = requests.get(booster_url)
tem_df = pd.json_normalize(temp_req.json())
#creating the dictionary with the booster versions#
booster_dict = {}
for ix in range(tem_df.shape[0]):
    booster_dict[tem_df.iloc[ix]["name"]] = tem_df.iloc[ix]["id"]
booster_dict = booster_dict
spacex_df = spacex_df.replace(list(booster_dict.values()),list(booster_dict.keys()))

Similarly we categorise all the payloads and the launchpad using their respective data sets.

In [8]:
payload_url = "https://api.spacexdata.com/v4/payloads"
payload_response = requests.get(payload_url)
payload = pd.json_normalize(payload_response.json())
payload_temp = payload[['mass_kg','orbit','id']]
launch_id, mass_kg, orbit = list(payload_temp.id), list(payload_temp.mass_kg), list(payload_temp.orbit)
#create a dictionary with orbits against payload ids
orbit_dict = {}
for orbit in list(set(payload_temp.orbit.values)):
    orbit_dict[orbit] = list(payload_temp[payload_temp['orbit']==orbit]['id'].values)
#create a dictionary with mass against payload ids
mass_dict = {}
for mass in list(set(payload_temp.mass_kg.values)):
    mass_dict[mass] = list(payload_temp[payload_temp['mass_kg']==mass]['id'].values)

In [9]:
spacex_df['orbit'] = spacex_df['payloads']
spacex_df['mass_kg'] = spacex_df['payloads']

In [10]:
for key in orbit_dict.keys():
    orbit_list = orbit_dict[key]
    spacex_df['orbit'] = spacex_df['orbit'].replace(orbit_list,key)
for key in mass_dict.keys():
    mass_list = mass_dict[key]
    spacex_df['mass_kg'] = spacex_df['mass_kg'].replace(mass_list,key)

In [11]:
spacex_df["orbit"] = spacex_df["orbit"].replace("5f839ac7818d8b59f5740d48","LEO")

In [12]:
spacex_df = spacex_df.drop("payloads",axis=1)

Getting the relevant classification and information for the launchpads from the relevant url

In [13]:
launch_url = "https://api.spacexdata.com/v4/launchpads"
launch_response = requests.get(launch_url)
launch_temp = pd.json_normalize(launch_response.json())
launch_temp = launch_temp[['name','id','latitude','longitude']]
temp = launch_temp.values
locations, ids = list(temp[:,0]), list(temp[:,1])
latitude, longitude = list(temp[:,2]), list(temp[:,3])
spacex_df['launchpad'] = spacex_df['launchpad'].replace(ids,locations)

In [14]:
spacex_df.launchpad.value_counts()

CCSFS SLC 40       72
KSC LC 39A         34
VAFB SLC 4E        16
Kwajalein Atoll     4
Name: launchpad, dtype: int64

In [15]:
spacex_df.reset_index(inplace=True)

The cores column has a lot of information and we want to use them for our analysis, so, there is a more detailed processing here. First, I will keep the keys of the core and then I will make the separate dataset for cores and then finally concat with the main data set.
To preserve the dignity of my progress, I will declare a temp_df and work with that.

In [16]:
temp_df = spacex_df.copy()

In [17]:
temp_df["core"] = temp_df["cores"].map(lambda x: x["core"]) #I will use this coloumn to anchor the core data to the main data
cores_df = pd.DataFrame(columns = list(spacex_df.cores[0].keys()))
for ix in range(spacex_df.shape[0]):
    cores_df = cores_df.append(spacex_df.cores.iloc[ix],ignore_index=True)
cores_df = cores_df.reset_index().drop('index',axis=1)


In [18]:
cores_df.head()

,core,flight,gridfins,legs,reused,landing_attempt,landing_success,landing_type,landpad
0,5e9e289df35918033d3b2623,1,False,False,False,False,None,None,None
1,5e9e289ef35918416a3b2624,1,False,False,False,False,None,None,None
2,5e9e289ef3591855dc3b2626,1,False,False,False,False,None,None,None
3,5e9e289ef359184f103b2627,1,False,False,False,False,None,None,None
4,5e9e289ef359185f2b3b2628,1,False,False,False,False,None,None,None


In [19]:
temp_df = pd.concat([spacex_df,cores_df],axis=1)
spacex_df = temp_df.drop(["index","cores","core"],axis=1)

Thus, we have the spacex data with the cores distionary expanded

In [20]:
spacex_df.head()

,rocket,launchpad,success,flight_number,date,orbit,mass_kg,flight,gridfins,legs,reused,landing_attempt,landing_success,landing_type,landpad
0,Falcon 1,Kwajalein Atoll,False,1,2006-03-24,LEO,20.0,1,False,False,False,False,None,None,None
1,Falcon 1,Kwajalein Atoll,False,2,2007-03-21,LEO,5eb0e4b6b6c3bb0006eeb1e2,1,False,False,False,False,None,None,None
2,Falcon 1,Kwajalein Atoll,True,4,2008-09-28,LEO,165.0,1,False,False,False,False,None,None,None
3,Falcon 1,Kwajalein Atoll,True,5,2009-07-13,LEO,200.0,1,False,False,False,False,None,None,None
4,Falcon 9,CCSFS SLC 40,True,6,2010-06-04,LEO,5eb0e4b7b6c3bb0006eeb1e7,1,False,False,False,False,None,None,None


In [21]:
temp_df = spacex_df.copy()
temp_df = pd.get_dummies(temp_df,columns = ["success","gridfins","legs","reused","landing_attempt","landing_success","landing_type"])

In [22]:
temp_df.head()

,rocket,launchpad,flight_number,date,orbit,mass_kg,flight,landpad,success_False,success_True,gridfins_False,gridfins_True,legs_False,legs_True,reused_False,reused_True,landing_attempt_False,landing_attempt_True,landing_success_False,landing_success_True,landing_type_ASDS,landing_type_Ocean,landing_type_RTLS
0,Falcon 1,Kwajalein Atoll,1,2006-03-24,LEO,20.0,1,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0
1,Falcon 1,Kwajalein Atoll,2,2007-03-21,LEO,5eb0e4b6b6c3bb0006eeb1e2,1,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0
2,Falcon 1,Kwajalein Atoll,4,2008-09-28,LEO,165.0,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0
3,Falcon 1,Kwajalein Atoll,5,2009-07-13,LEO,200.0,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0
4,Falcon 9,CCSFS SLC 40,6,2010-06-04,LEO,5eb0e4b7b6c3bb0006eeb1e7,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0


In [23]:
spacex_df = temp_df.copy()

In [24]:
spacex_df["launch_lat"] = spacex_df["launchpad"].replace(locations,latitude)
spacex_df["launch_long"] = spacex_df["launchpad"].replace(locations,longitude)

In [25]:
#initialised the claffication variables
spacex_df.head()

,rocket,launchpad,flight_number,date,orbit,mass_kg,flight,landpad,success_False,success_True,gridfins_False,gridfins_True,legs_False,legs_True,reused_False,reused_True,landing_attempt_False,landing_attempt_True,landing_success_False,landing_success_True,landing_type_ASDS,landing_type_Ocean,landing_type_RTLS,launch_lat,launch_long
0,Falcon 1,Kwajalein Atoll,1,2006-03-24,LEO,20.0,1,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
1,Falcon 1,Kwajalein Atoll,2,2007-03-21,LEO,5eb0e4b6b6c3bb0006eeb1e2,1,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
2,Falcon 1,Kwajalein Atoll,4,2008-09-28,LEO,165.0,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
3,Falcon 1,Kwajalein Atoll,5,2009-07-13,LEO,200.0,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
4,Falcon 9,CCSFS SLC 40,6,2010-06-04,LEO,5eb0e4b7b6c3bb0006eeb1e7,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,28.561857,-80.577366


In [26]:
#here we classify the landpad of the data
spacex_df.landpad.value_counts()

5e9e3032383ecb6bb234e7ca    50
5e9e3033383ecbb9e534e7cc    23
5e9e3032383ecb267a34e7c7    15
5e9e3032383ecb554034e7c9     3
5e9e3033383ecb075134e7cd     3
5e9e3032383ecb761634e7cb     2
Name: landpad, dtype: int64

In [27]:
landpad_url= "https://api.spacexdata.com/v4/landpads"
landpad_req = requests.get(landpad_url)
landpad_df = pd.json_normalize(landpad_req.json())
landpad_df = landpad_df[["id","locality","latitude","longitude"]]
temp_df = spacex_df.copy()
temp_df["landpad"] = temp_df["landpad"].replace(list(landpad_df["id"]),list(landpad_df["locality"]))

In [28]:
temp_df["landpad"].value_counts()

Port of Los Angeles          50
Port Canaveral               28
Cape Canaveral               15
Vandenberg Air Force Base     3
Name: landpad, dtype: int64

In [29]:
temp_df.head()

,rocket,launchpad,flight_number,date,orbit,mass_kg,flight,landpad,success_False,success_True,gridfins_False,gridfins_True,legs_False,legs_True,reused_False,reused_True,landing_attempt_False,landing_attempt_True,landing_success_False,landing_success_True,landing_type_ASDS,landing_type_Ocean,landing_type_RTLS,launch_lat,launch_long
0,Falcon 1,Kwajalein Atoll,1,2006-03-24,LEO,20.0,1,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
1,Falcon 1,Kwajalein Atoll,2,2007-03-21,LEO,5eb0e4b6b6c3bb0006eeb1e2,1,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
2,Falcon 1,Kwajalein Atoll,4,2008-09-28,LEO,165.0,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
3,Falcon 1,Kwajalein Atoll,5,2009-07-13,LEO,200.0,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,9.047721,167.743129
4,Falcon 9,CCSFS SLC 40,6,2010-06-04,LEO,5eb0e4b7b6c3bb0006eeb1e7,1,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,28.561857,-80.577366


In [30]:
landpad_df[landpad_df["locality"]=="Port Canaveral"] 
#checking to confirm if the repeated ids are correct, and they match, so the matching has been accurate

,id,locality,latitude,longitude
4,5e9e3032383ecb761634e7cb,Port Canaveral,28.410400,-80.618800
5,5e9e3033383ecbb9e534e7cc,Port Canaveral,28.410400,-80.618800
6,5e9e3033383ecb075134e7cd,Port Canaveral,33.729186,-118.262015


In [31]:
spacex_df = temp_df.copy()

In [32]:
spacex_df.tail()

,rocket,launchpad,flight_number,date,orbit,mass_kg,flight,landpad,success_False,success_True,gridfins_False,gridfins_True,legs_False,legs_True,reused_False,reused_True,landing_attempt_False,landing_attempt_True,landing_success_False,landing_success_True,landing_type_ASDS,landing_type_Ocean,landing_type_RTLS,launch_lat,launch_long
121,Falcon 9,KSC LC 39A,135,2021-09-16,LEO,607a382f5a906a44023e0867,3,Port Canaveral,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,28.608058,-80.603956
122,Falcon 9,KSC LC 39A,136,2021-11-11,ISS,12519.0,2,Port Canaveral,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,28.608058,-80.603956
123,Falcon 9,CCSFS SLC 40,137,2021-11-13,LEO,13260.0,9,Port Canaveral,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,28.561857,-80.577366
124,Falcon 9,VAFB SLC 4E,138,2021-11-24,TLI,670.0,2,Port of Los Angeles,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,34.632093,-120.610829
125,Falcon 9,CCSFS SLC 40,139,2021-12-01,PO,13260.0,9,Port Canaveral,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,28.561857,-80.577366


In [33]:
#ordering the columns to make sense of the data better
ordered_cols = ["date","rocket","launchpad","launch_lat","launch_long","flight_number","flight","orbit","mass_kg","landpad"]+list(spacex_df.columns)[8:-2]
spacex_df = spacex_df[ordered_cols]
spacex_df.head()

,date,rocket,launchpad,launch_lat,launch_long,flight_number,flight,orbit,mass_kg,landpad,success_False,success_True,gridfins_False,gridfins_True,legs_False,legs_True,reused_False,reused_True,landing_attempt_False,landing_attempt_True,landing_success_False,landing_success_True,landing_type_ASDS,landing_type_Ocean,landing_type_RTLS
0,2006-03-24,Falcon 1,Kwajalein Atoll,9.047721,167.743129,1,1,LEO,20.0,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0
1,2007-03-21,Falcon 1,Kwajalein Atoll,9.047721,167.743129,2,1,LEO,5eb0e4b6b6c3bb0006eeb1e2,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0
2,2008-09-28,Falcon 1,Kwajalein Atoll,9.047721,167.743129,4,1,LEO,165.0,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0
3,2009-07-13,Falcon 1,Kwajalein Atoll,9.047721,167.743129,5,1,LEO,200.0,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0
4,2010-06-04,Falcon 9,CCSFS SLC 40,28.561857,-80.577366,6,1,LEO,5eb0e4b7b6c3bb0006eeb1e7,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0


 With a satisfactory level of data preparation complete, this dataset is to be exported in csv format for analysis.

In [34]:
spacex_df.to_csv("spacex_df_cap.csv", index=False)

In [35]:
cores_df["landing_success"].value_counts()

True     91
False    10
Name: landing_success, dtype: int64

In [36]:
spacex_df["mass_kg"] =  pd.to_numeric(spacex_df['mass_kg'],errors='coerce')

In [37]:
meanmass = list(spacex_df.groupby("rocket").mean()["mass_kg"].values) #ix_0 is F1 and ix_1 is F9

In [38]:
meanmass = [round(x,2) for x in meanmass]

In [39]:
meanmass

[128.33, 7341.2]

In [40]:
#Columns renaming#
newcols = {"mass_kg":"Payload (Kg)", "landing_success_False":"Failed Landing","landing_success_True":"Successful Landing",
           "landing_type_ASDS":"Landing ASDS","landing_type_RTLS":"Landing RTLS","landing_type_Ocean":"Landing Ocean",
           "date":"Date","rocket":"Rocket","launchpad":"Launch Site", "launch_lat":"Launch Latitude",
           "launch_long":"Launch Longitude","flight_number":"Flight Number","flight":"Flight","orbit":"Orbit",
           "success_False":"Failed Launch","success_True":"Successful Launch","landpad":"Land Site"}

In [41]:
spacex_df_pretty = spacex_df.rename(columns=newcols)
spacex_df_pretty["Year"]=pd.to_datetime(spacex_df_pretty["Date"]).dt.year

In [42]:
#### SET THE MEAN PAYLOAD VALUES ####

In [43]:
for row in range(spacex_df_pretty.shape[0]):
    rocket, mass = spacex_df_pretty.loc[row,"Rocket"],spacex_df_pretty.loc[row,"Payload (Kg)"]
#     print(rocket,", ",mass)
#     print(np.isnan(mass))
    if np.isnan(mass) == True:
        if rocket == "Falcon 1": spacex_df_pretty.loc[row,"Payload (Kg)"] = meanmass[0]
        if rocket == "Falcon 9": spacex_df_pretty.loc[row,"Payload (Kg)"] = meanmass[1]
    else:
        spacex_df_pretty.loc[row,"Payload (Kg)"] = mass

In [44]:
spacex_df_pretty.head(20)

,Date,Rocket,Launch Site,Launch Latitude,Launch Longitude,Flight Number,Flight,Orbit,Payload (Kg),Land Site,Failed Launch,Successful Launch,gridfins_False,gridfins_True,legs_False,legs_True,reused_False,reused_True,landing_attempt_False,landing_attempt_True,Failed Landing,Successful Landing,Landing ASDS,Landing Ocean,Landing RTLS,Year
0,2006-03-24,Falcon 1,Kwajalein Atoll,9.047721,167.743129,1,1,LEO,20.00,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,2006
1,2007-03-21,Falcon 1,Kwajalein Atoll,9.047721,167.743129,2,1,LEO,128.33,None,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,2007
2,2008-09-28,Falcon 1,Kwajalein Atoll,9.047721,167.743129,4,1,LEO,165.00,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,2008
3,2009-07-13,Falcon 1,Kwajalein Atoll,9.047721,167.743129,5,1,LEO,200.00,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,2009
4,2010-06-04,Falcon 9,CCSFS SLC 40,28.561857,-80.577366,6,1,LEO,7341.20,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,2010
5,2012-05-22,Falcon 9,CCSFS SLC 40,28.561857,-80.577366,8,1,LEO,525.00,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,2012
6,2013-03-01,Falcon 9,CCSFS SLC 40,28.561857,-80.577366,10,1,ISS,677.00,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,2013
7,2013-09-29,Falcon 9,VAFB SLC 4E,34.632093,-120.610829,11,1,PO,500.00,None,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,2013
8,2013-12-03,Falcon 9,CCSFS SLC 40,28.561857,-80.577366,12,1,GTO,3170.00,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,2013
9,2014-01-06,Falcon 9,CCSFS SLC 40,28.561857,-80.577366,13,1,GTO,3325.00,None,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,2014


In [45]:
spacex_df_pretty.to_csv("spacex_df_pretty.csv")